In this demo, we demonstrate how to use the MaxEnt_Reg script. We will be going over how to do the following:

1. Reading in the MaxEnt data and constraint information
2. Running the model using different gradient descent techniques (without regularization)
3. How to add regularization (target prior ```TGTPrior``` vs. difference prior ```DIFPrior```)

In [4]:
## Good old NumPy
import numpy as np

## Specialized classes
from utils.MaxEnt import MaxEnt
from utils.Regularization import TGTPrior, DIFPrior

## Utility function for reading in constraint and data information
from utils.OTSoft_file_reader import get_info


For this demonstration, we will be looking at the Hayes Pseudo-Korean data

In [5]:
## Read in data and constraint information
constraint_names, underlying_forms, candidates, violations, observed_probs = get_info(
    "toy_datasets/HayesPseudoKorean-RichBase.txt"
)

## Initialize a MaxEnt object
me = MaxEnt(constraint_names)

print(f"Initial weights: {me.cws}")


Initial weights: [[0.30245549]
 [4.04179978]
 [9.41291646]
 [0.71998226]
 [6.67540665]
 [0.21998625]
 [4.25873865]
 [4.66374388]]


In [6]:
## Perform learning without regularization and returns the final weights
new_weights, learning_history = me.SGD_learn(violations=violations, observed_prob=observed_probs)

print(f"Weights after learning: {new_weights}")


Weights after learning: [[0.79169523]
 [4.59982931]
 [9.7173913 ]
 [1.27396174]
 [6.17693327]
 [0.74193051]
 [4.76444165]
 [4.54408798]]
